In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
#Template Matching

In [27]:
images=['1.jpg','2.jpg','3.jpg','4.jpg','5.jpg','6.jpg','7.jpg','8.jpg','9.jpg','10.jpg']
templates = ['1 face.jpg','1 eyes.jpg', '1 nose.jpg']
methods = ['cv2.TM_CCOEFF', 'cv2.TM_CCOEFF_NORMED', 'cv2.TM_SQDIFF', 'cv2.TM_SQDIFF_NORMED']

for templ in templates:
    tmp = cv2.imread(templ,0)
    w, h = tmp.shape[::-1]
    for image in images:
        img = cv2.imread(image,0)
        template_matching(img, tmp, images.index(image), templates.index(templ))

In [19]:
def template_matching(image, template, n, m):
    for meth in methods:
        method = eval(meth)
        img = image.copy()

        res = cv2.matchTemplate(img, template, method)
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)

        if method in [cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]:
            top_left = min_loc
        else:
            top_left = max_loc
        bottom_right = (top_left[0] + w, top_left[1] + h)
        
        cv2.rectangle(img, top_left, bottom_right, (255, 0, 0), 2)

        plt.imshow(img, cmap='gray')
        plt.title(meth)

        figure_name = "aares-" + str(n+1) + "-" + str(m+1) + "-" + str(method)
        plt.savefig(figure_name)

In [ ]:
#Face_symmetry

In [138]:
def face_symmetry(face_cropped):
        size = face_cropped.shape
        w = int(size[1]/10)
        if w < 3:
            w = 3
        line = w
        best_central = [w, 1000000]
        while line < size[1] - w:
            tmp = 0
            for i in range(1, w+1):
                for j in range(size[0]):
                    tmp += np.abs(int(face_cropped[j][line-i]) - int(face_cropped[j][line+i]))
            if tmp < best_central[1]:
                best_central = [line, tmp]
            line += 2
        w1 = int(w/2)
        if w1 < 3:
            w1 = 3
        best_left = [w1, 1000000]
        best_right = [best_central[0] + w1, 1000000]
        line1 = w1
        line2 = best_central[0] + w1
        while line1 < best_central[0] - w1 and line2 < size[1] - w1:
            tmp1 = 0
            tmp2 = 0
            for i in range(1, w1+1):
                for j in range(int(size[0]/4), int(3*size[0]/4)):
                    tmp1 += np.abs(int(face_cropped[j][line1-i]) - int(face_cropped[j][line1+i]))
                    tmp2 += np.abs(int(face_cropped[j][line2-i]) - int(face_cropped[j][line2+i]))
            if tmp1 < best_left[1]:
                best_left = [line1, tmp1]
            if tmp2 < best_right[1]:
                best_right = [line2, tmp2]
            line1 += 2
            line2 += 2
        return best_central[0], best_left[0], best_right[0]

In [ ]:
# Viola-Jones

In [154]:
images = ['1.jpg','3.jpg', '4.jpg', '7.jpg', '8.jpg', '9.jpg', '11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg', '16.jpg', '17.jpg', '19.jpg', '20.jpg', '21.jpg', '22.jpg']

def convertToRGB(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)


def faceDetection (image, image_gray, n):
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    faces_rects = face_cascade.detectMultiScale(image_gray, scaleFactor=1.2, minNeighbors=5)

    eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')
    eye_rects = eye_cascade.detectMultiScale(image_gray, scaleFactor=1.3, minNeighbors=7)

    for (x, y, w, h) in faces_rects:
        cv2.rectangle(image, (x, y), (x+w, y+h), (0, 255, 0), 5)
        borders = int(w/10)
        c, l, r = face_symmetry(image_gray[y:y+h, x+borders:x+w-borders])
        print(c, l, r)
        cv2.line(image, (x+borders+l, y), (x+borders+l, y+h), (255,255,255), 5)
        cv2.line(image, (x+borders+c, y), (x+borders+c, y+h), (255,0,255), 5) 
        cv2.line(image, (x+borders+r, y), (x+borders+r, y+h), (255,255,255), 5)
            
        for (ex,ey,ew,eh) in eye_rects:
            cv2.rectangle(image, (ex,ey), (ex+ew, ey+eh), (255,0,0), 5)
    
    #convert image to RGB and show image
    plt.imshow(convertToRGB(image))
    figure_name = "VJ-" + str(n+1) + "-face"
    plt.savefig(figure_name)


for img in images:
    image = cv2.imread(img)
    # Converting to grayscale
    image_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Displaying the grayscale image
    plt.imshow(image_gray, cmap='gray')
    faceDetection(image, image_gray, images.index(img))